# W-Projection with Oversampling - Simple Test

A tweakable end-to-end runthrough. Tests W-Projection with oversampling.

In [ ]:
import numpy as np
import logging
import os

import astropy.units as u
import astropy.constants as const
import fastimgproto.imager as imager
from fastimgproto.gridder.gridder import convolve_to_grid
import fastimgproto.visibility as visibility

from astropy.coordinates import Angle, SkyCoord, AltAz, EarthLocation
from astropy.time import Time
from fastimgproto.gridder.conv_funcs import GaussianSinc
from fastimgproto.skymodel.helpers import SkyRegion, SkySource
from fastimgproto.sourcefind.image import SourceFindImage
from fastimgproto.telescope.readymade import Meerkat

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# Plot image pixels in cartesian ordering (i.e. y-positive == upwards):
plt.rcParams['image.origin'] = 'lower'
# Make plots bigger
plt.rcParams['figure.figsize'] = 10, 10

## Set up the telescope

In [ ]:
telescope = Meerkat()

In [ ]:
pointing_centre = SkyCoord(0 * u.deg, -30 * u.deg)
obs_central_frequency = 30 * u.MHz
wavelength = const.c / obs_central_frequency
transit_time = telescope.next_transit(pointing_centre.ra,
                                      start_time=Time('2017-01-01'))

Choose the observation times / sampling regime

In [ ]:
nstep=10
obs_times = transit_time + np.linspace(-4, 4, nstep) * u.hr
print("Generating UVW-baselines for {} timesteps".format(nstep))
uvw_m = telescope.uvw_tracking_skycoord(pointing_centre, obs_times)
# From here on we use UVW as multiples of wavelength, lambda:
uvw_lambda = (uvw_m / wavelength).to(u.dimensionless_unscaled).value

## Configure the sources to simulate

Sources

In [ ]:
source_posn = SkyCoord(
    ra=pointing_centre.ra - 10.0 * u.deg,
    dec=pointing_centre.dec - 0.0 * u.deg)

sources = [
    SkySource(position=source_posn, flux=1 * u.Jy),
]

In [ ]:
all_sources = sources

## Simulate visibilities, given the source-list and UVW baseline info:

In [ ]:
# Simulate incoming data; includes transient sources, noise:
baseline_noise_level = 0.001 * u.Jy

data_vis = visibility.visibilities_for_source_list(
    pointing_centre,
    source_list = all_sources, 
    uvw = uvw_lambda)

#data_vis = visibility.add_gaussian_noise(baseline_noise_level, data_vis)

snr_weights = np.ones_like(data_vis,dtype=np.float_)

## Configure and run the imager

In [ ]:
image_size=1024 * u.pixel
cell_size=100 * u.arcsecond

In [ ]:
kernel_support = 3
kernel_func = GaussianSinc(trunc=kernel_support)

## Pipeline

Run pipeline using several oversampling values

In [ ]:
rms_list = []
peakflux_list = []
extremum_list = []
oversampling_list = [1, 2, 4, 6, 8, 10, 12, 14, 16]

for oversampling in oversampling_list:
    image, beam = imager.image_visibilities(
        data_vis,
        snr_weights,
        uvw_lambda,
        image_size=image_size,
        cell_size=cell_size,
        kernel_func=kernel_func,
        kernel_support=kernel_support,
        kernel_exact=False,
        kernel_oversampling=oversampling,
        num_wplanes=32,
        max_wpconv_support=30,
        )
    image = np.real(image)
    beam = np.real(beam)
    
    detection_n_sigma=50
    analysis_n_sigma=50
    sfimage = SourceFindImage(data=np.real(image),
                              detection_n_sigma=detection_n_sigma,
                              analysis_n_sigma=analysis_n_sigma,
                              )
    
    rms_list.append(sfimage.rms_est)
    isl = sfimage.islands[0]
    peakflux_list.append(isl.params.extremum.value)
    extremum_list.append(isl.params.extremum)

## Analysis of Results

In [ ]:
rms_list

In [ ]:
peakflux_list

In [ ]:
extremum_list

### Plot peak-flux as a function of oversampling

In [ ]:
%matplotlib inline

plt.plot(oversampling_list, peakflux_list)
plt.ylabel('Peak flux [Jy]')
plt.xlabel('Oversampling')
plt.show()


### Plot dynamic range as a function of oversampling

In [ ]:
%matplotlib inline

plt.plot(oversampling_list, np.array(peakflux_list) / np.array(rms_list))
plt.ylabel('Dynamic Range')
plt.xlabel('Oversampling')
plt.show()

### Plot rms estimation as a function of oversampling

In [ ]:
%matplotlib inline

plt.plot(oversampling_list, rms_list)
plt.ylabel('RMS estimation')
plt.xlabel('Oversampling')
plt.show()